# <u>Métodos No Supervisados - Parte 1</u>

## Segmentación K-Means

### Caso:
En una entidad bancaria existen varios canales de comunicación tales como: ATM, Oficinas, IVR, Banca por internet-Móvil, etc.
Sin embargo, al realizar las comunicaciones de ofertas a los clientes de dicha entidad bancaria, el cliente recibe diversas ofertas de distintos canales sin saber si les da importancia o no, por lo que ya interviene un gasto por parte de la entidad, ya que, realiza alianzas estratégicas de campaña.


Por tanto: Se requiere identificar cuál sería el medio de comunicación preferido para los clientes y así enviarles ofertas, advertencia, recordatorios, etc. más direccionadas.

### 1. Librerias

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

%matplotlib inline
plt.rcParams['figure.figsize'] = (7, 4)
plt.style.use('ggplot')

### 2. Extracción Base de datos

In [ ]:
dataFrame = pd.read_csv("data/01dataBaseMulti.txt",delimiter='|')
print(dataFrame.head())

In [ ]:
print("Número de filas: " + str(dataFrame.shape[0]))
print("Número de columnas: " + str(dataFrame.shape[1]))

In [ ]:
dataFrame.info()

### 3. Metodología

In [ ]:
#### 3.1 Análisis Previo (objetivo)
#### 3.2 Exploración (descriptivo, grafico barras,cajas)
#### 3.3 Transformación (standarización,cajas)
#### 3.4 Outliers (analisis y eliminación de outliers)
#### 3.5 Dimensionamiento (PCA)
#### 3.6 Modelamiento
#### 3.7 Evaluación
#### 3.8 Perfilamiento
#### 3.9 Visualización

In [ ]:
# Variables objetivo de estudio:
channelName = ['trxAplus', 'trxBcaex', 'trxSalex', 'trxBm', 'trxBxi', 'trxIvr', 'trxSbt', 'trxVent',
               'trxAtm','trxPostc', 'trxPostd']

#### 3.1 Análisis Negocio

El dataset ya se encuentra trabajado a nivel de cliente con sus respectivas variables, se consideraron filtros de criterios de autoasignados, distribución histórica de transacciones, etc. 

#### 3.2 Exploración

In [ ]:
print(dataFrame[channelName].describe())

In [ ]:
dataFrame[channelName].hist(bins = 50, figsize=(20,15))
plt.show()

In [ ]:
# Gráfico de cajas por variable en estudio:
for columnName in channelName:
    plt.title(columnName)  
    plt.boxplot(dataFrame[columnName], 0, 'gD')    
    plt.show()

#### 3.3 Transformación

In [ ]:
# guardamos la data original para poder usar las variables al final
data_orig = dataFrame.copy()

In [ ]:
# ------------------------------
# Creamos el objeto para escalar
# ------------------------------
from sklearn import preprocessing

scaler = preprocessing.StandardScaler()
#scaler = preprocessing.MinMaxScaler(feature_range=(0, 1))

# ************
# Lo aplicamos
# ************
for columnName in channelName:
    dataFrame[columnName] = scaler.fit_transform(dataFrame[columnName].values.reshape(-1, 1))

#for columnName in columName:
#    dataFrame[columnName] = dataFrame[columnName]/dataFrame['trx']

In [ ]:
dataFrame[channelName].head()

#### 3.4 Outliers

In [ ]:
# Cálculo de intervalo del diagrama de cajas - Método de Rango Intercuartílico
#def calculateNumOutliars(serie):
#  Q01 = serie.quantile(0.25)
#  Q03 = serie.quantile(0.75)
#  IQR = Q03 - Q01
#  a = (serie < (Q01 - 1.5 * IQR)) | (serie > (Q03 + 1.5 * IQR))
#  numOutliars = a[a == True].shape[0]
#  return numOutliars

In [ ]:
# Usamos el método de Z-score (considerando se distribuye Normalmente) --- para grandes volúmenes de datos
def calculateNumOutliars(serie):
    mu = serie.mean()
    desv = np.std(serie)
    a = ((serie-mu)/desv < -3) | ((serie-mu)/desv > 3)
    numOutliars = a[a == True].shape[0]
    return a,numOutliars    

In [ ]:
numTotal = dataFrame.shape[0]
for columnName in channelName:
    a,numOutliars = calculateNumOutliars(dataFrame[columnName])
    # Creamos nuevos campos para filtrar los Outliers 
    dataFrame['flg_'+columnName]=a
    print('*'+columnName)
    if numOutliars > 0:
        print("Número de valores outliars: " + str(numOutliars))
        print("Porcentaje: " + str(np.round(numOutliars * 100 / numTotal, 2)) + "%")
    else:
        print("****No hay Outliers")    
    print("\n")

In [ ]:
dataFrame.head()

In [ ]:
# ************************
# Extrayendo los Outliers
# ************************
# Luego que cada variable tenga menos del 10% de Outlier, se filtra de manera Multivariada (este filtro podría ser
# considerado como un segmento Heavy)

dataFrame = dataFrame[(dataFrame['flg_trxAplus']==False)&
                      (dataFrame['flg_trxBcaex']==False)&
                      (dataFrame['flg_trxSalex']==False)&
                      (dataFrame['flg_trxBm']==False)&
                      (dataFrame['flg_trxBxi']==False)&
                      (dataFrame['flg_trxIvr']==False)&
                      (dataFrame['flg_trxSbt']==False)&
                      (dataFrame['flg_trxVent']==False)&
                      (dataFrame['flg_trxAtm']==False)&
                      (dataFrame['flg_trxPostc']==False)&
                      (dataFrame['flg_trxPostd']==False)]

In [ ]:
# acotamos la data original a la base sin outliers
data_orig = data_orig.iloc[dataFrame.index,]
data_orig = data_orig.reset_index()

In [ ]:
# Refrescamos los índices del Data frame final
dataFrame = dataFrame.reset_index()
print('Cantidad de Registros sin Outliers: '+str(dataFrame.shape[0]))
dataFrame.head()

#### 3.5 Reducción de dimensión (PCA)

In [ ]:
# Calculamos el máximo número de componentes (Nro variables = Nro máximo de componentes)
from sklearn.decomposition import PCA

pca = PCA()
pca.fit(dataFrame[channelName])
pca.explained_variance_ratio_

In [ ]:
for i in range(len(pca.components_)):
    print('% Var. explicada ('+str(i+1)+' componentes): ', np.cumsum(pca.explained_variance_ratio_)[i]*100)
    
plt.bar(range(1,len(pca.components_)+1),pca.explained_variance_ratio_, alpha=.2,color='0')
plt.plot(range(1,len(pca.components_)+1),np.cumsum(pca.explained_variance_ratio_),alpha=0.4)
plt.title("Varianza explicada y pareto")
plt.show()

Según el gráfico podemos observar que hay tendencia a que cada componente aporta información relevante, por lo que no existe alguna relación fuerte entre variables

In [ ]:
# Elegimos la componente adecuada:
pcaFin = PCA(n_components=11)
pcaFin.fit(dataFrame[channelName])
pd.DataFrame(pcaFin.components_,columns=channelName)

Confirmamos nuestra evidencia en el gráfico de Pareto, donde: Para cada variable está asignado a cada Componente, Por tanto no existe reducción de dimensiones para nuestro estudio de canales de transacción

#### 3.6 Modelamiento

In [ ]:
from sklearn.cluster import KMeans
from sklearn import metrics
from sklearn.metrics import pairwise_distances_argmin_min 

In [ ]:
# Calculando el número de clúster adecuado:
X = dataFrame[channelName]

numClus = range(1, 20)
kmeans = [KMeans(n_clusters=i,max_iter=600, algorithm = 'auto') for i in numClus]
kmeans
score = [kmeans[i].fit(X).score(X)*-1 for i in range(len(kmeans))]
score
plt.plot(numClus,score)
plt.xlabel('Número de Clúster')
plt.ylabel('Score')
plt.title('Curva de Inflexión')
plt.show()

In [ ]:
# Nos fijamos de los indicadores de clustering:

ctdDf = int(0.1*dataFrame.shape[0])
cluster = [kmeans[i].predict(X) for i in range(len(kmeans))]

for i in range(1,11):    
    print(str(i+1)+' clústeres:')
    print('Inercia: '+str(kmeans[i].inertia_))
    print('Silueta: '+str(metrics.silhouette_score(X, cluster[i], metric='euclidean',sample_size=ctdDf)))
    print("\n")

Visualizando los grupos en 2-D para tener alguna noción de como se agrupan, en esta ocasión probaremos distintos par de variables

In [ ]:
fig = plt.figure()
f1 = dataFrame['trxAtm'].values
f2 = dataFrame['trxBm'].values
 
#colores=['red','green','blue','cyan','yellow']
colores=['red','green','blue']
asignar=[]
for row in cluster[1]:
    asignar.append(colores[row])
    
plt.scatter(f1, f2, c=asignar, s=20)
#plt.scatter(centroide[2][:, 0], centroide[2][:, 1], marker='*', c='yellow', s=100)
plt.show()

In [ ]:
fig = plt.figure()
f1 = dataFrame['trxAtm'].values
f2 = dataFrame['trxPostd'].values
 
#colores=['red','green','blue','cyan','yellow']
colores=['red','green','blue']
asignar=[]
for row in cluster[2]:
    asignar.append(colores[row])
    
plt.scatter(f1, f2, c=asignar, s=20)
#plt.scatter(centroide[2][:, 0], centroide[2][:, 1], marker='*', c='yellow', s=100)
plt.show()

#### 3.7 Evaluación

In [ ]:
numClus = [5,6,7,8]

In [ ]:
centroide = [kmeans[i].cluster_centers_ for i in range(len(kmeans))]
copy =  pd.DataFrame()

for i in numClus:
    # Distribución de los grupos por clúster:
    copy['cluster'] = cluster[i-1]
    cantidadGrupo =  pd.DataFrame()
    cantidadGrupo['ctdCliente']=copy.groupby('cluster').size()
    cantidadGrupo['pctCliente']=round(100*cantidadGrupo['ctdCliente']/cantidadGrupo['ctdCliente'].sum(),2)
    
    # gráfico de los grupos según su distribución:
    plt.pie(cantidadGrupo['pctCliente'], labels=cantidadGrupo.index, autopct='%1.1f%%')
    plt.title('Clúster '+str(i))
    plt.legend()
    plt.show()
    print(cantidadGrupo)       
    print('\n')

In [ ]:
numClusFinal = int(input('Ingrese el número de clúster: '))

In [ ]:
kmeans_fin = KMeans(n_clusters=numClusFinal,max_iter=600, algorithm = 'auto').fit(X)
cluster = pd.DataFrame(kmeans_fin.predict(X))
cluster.columns = ['cluster']

centroide = kmeans_fin.cluster_centers_
# Distribución de los grupos por clúster:
cantidadGrupo =  pd.DataFrame()
cantidadGrupo['ctdCliente']=cluster.groupby('cluster').size()
cantidadGrupo['pctCliente']=round(100*cantidadGrupo['ctdCliente']/cantidadGrupo['ctdCliente'].sum(),2)

# gráfico de los grupos según su distribución:
plt.pie(cantidadGrupo['pctCliente'], labels=cantidadGrupo.index, autopct='%1.1f%%')
plt.title('Clúster '+str(kmeans_fin))
plt.legend()
plt.show()
print(cantidadGrupo)       
print('\n')

In [ ]:
dfTransp = pd.DataFrame(centroide,columns=channelName).T
corr = dfTransp.corr()

def plot_correlations(corr):
    sns.set(style="white")
    cmap = sns.diverging_palette(220, 10, as_cmap=True)
    mask = np.zeros_like(corr, dtype=bool)
    mask[np.triu_indices_from(mask)] = True
    sns.heatmap(abs(corr), mask=mask, cmap=cmap, vmax=1, center=0,square=True) 
    
plot_correlations(corr)

In [ ]:
dfTransp

In [ ]:
# revisando los promedios en la data original
data_orig['cluster'] = cluster
data_orig.groupby('cluster')[channelName].mean().T

#### 3.8 Perfilamiento

In [ ]:
dfTransp.plot(figsize=(20,20))
plt.show()